# SET (grib file)

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
%%capture
!conda update conda

In [ ]:
!conda --version
!which conda

conda 4.11.0
/usr/local/bin/conda


In [ ]:
#%%capture
!conda install metview  -c conda-forge
!conda install metview-python  -c conda-forge

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - metview


The following NEW packages will be INSTALLED:

  cudatoolkit        conda-forge/linux-64::cudatoolkit-11.1.1-h6406543_10


Preparing transaction: / done
Verifying transaction: \ done
Executing transaction: / - \ | By downloading and using the CUDA Toolkit conda packages, you accept the terms and conditions of the CUDA End User License Agreement (EULA): https://docs.nvidia.com/cuda/eula/index.html

done
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

# All requested packages already installed.



In [ ]:
!pip install opencage

In [ ]:
# import os
# os._exit(00)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import metview as mv
import numpy as np
import pandas as pd
#from opencage.geocoder import OpenCageGeocode
from tqdm import tqdm

In [ ]:
!wget 'https://raw.githubusercontent.com/hohohohia/misc/main/county_coord.npy'
!unzip 'drive/MyDrive/HWs/SETdaily.zip' -d './'
county_list_2d=pd.read_csv('drive/MyDrive/HWs/County_450.csv')
from google.colab import drive
drive.flush_and_unmount()

--2022-03-12 04:59:58--  https://raw.githubusercontent.com/hohohohia/misc/main/county_coord.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 172928 (169K) [application/octet-stream]
Saving to: ‘county_coord.npy’

county_coord.npy    100%[===================>] 168.88K  --.-KB/s    in 0.02s   

2022-03-12 04:59:58 (7.28 MB/s) - ‘county_coord.npy’ saved [172928/172928]

Archive:  drive/MyDrive/HWs/SETdaily.zip
  inflating: ./SETdaily.grib         


In [ ]:
filename = './SETdaily.grib'
if mv.exist(filename):
    wg = mv.read(filename)
else:
    wg = mv.gallery.load_dataset(filename)
print(wg)
print(len(wg))

Fieldset (5114 fields)
5114


In [ ]:
county_list=county_list_2d.county.to_numpy()
county_list[:5]

array([1007, 1067, 1077, 1095, 1099])

In [ ]:
def county_coords_getter(county_list=county_list):
  county_dic=pd.read_csv('https://raw.githubusercontent.com/hohohohia/misc/main/fips.csv',delimiter=',')
  county_dic=county_dic.to_numpy()
  county_dic={i[0]:str(i[1])+', '+str(i[2]) for i in county_dic}
  target=[[j,county_dic[j]] for j in county_list]
  key = 'e969ca058ef8475691c5d75f9e1c20fa'
  geocoder = OpenCageGeocode(key)
  target_coords=[]
  for i in tqdm(range(len(target))):
    query = target[i][1]
    results = geocoder.geocode(query)
    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']
    target_coords.append([query,lat,lng])
    #if i==20: break
  return target_coords

In [ ]:
toggle=False
if toggle==True: 
  target_coords=county_coords_getter()
else:
  target_coords=np.load('county_coord.npy')

In [ ]:
print(target_coords[:5])
coords=[[i[1],i[2]] for i in target_coords]

[['Bibb County, AL' '32.971079' '-87.1227088']
 ['Henry County, AL' '31.4838041' '-85.2447377']
 ['Lauderdale County, AL' '34.9049555' '-87.6747398']
 ['Marshall County, AL' '34.3396046' '-86.3301801']
 ['Monroe County, AL' '31.5593935' '-87.3781004']]


In [ ]:
def ts_gen(coord,df=False):
  roff = np.array(mv.nearest_gridpoint(wg['ro'], coord))
  evap = np.array(mv.nearest_gridpoint(wg['e'], coord))
  times = mv.valid_date(wg['ro'])
  if df==True:
    df=pd.DataFrame(zip(times,roff,evap),columns=['time','run_off','evaporation'])
    return times, roff, evap, df
  else: 
    return roff, evap

def mat_gen(coords):
  result=[]
  v=0
  for i in tqdm(coords):
    if v==0:
      i=[float(i[0]),float(i[1])]
      print(i)
      result.append(mv.valid_date(wg['ro']))
      cache1=ts_gen(i)
      result.append(cache1[0])
      result.append(cache1[1])
    else: 
      i=[float(i[0]),float(i[1])]
      cache1=ts_gen(i)
      result.append(cache1[0])
      result.append(cache1[1])
    v=1
  return result

In [ ]:
matrix_=mat_gen(coords)

  0%|          | 0/450 [00:00<?, ?it/s]

[32.971079, -87.1227088]


 25%|██▌       | 114/450 [17:27:44<54:21:00, 582.32s/it]

In [ ]:
column_name=[]
v=0
for i in county_list:
  if v==0: 
      column_name.append('time')
  for j in range(2):
      if j==0: column_name.append(str(i)+'_roff')
      else: column_name.append(str(i)+'_evap')
  v=1

In [ ]:
cache=pd.DataFrame(data=np.transpose(matrix_),columns=column_name)
cache.to_csv('SET.csv')